In [20]:
import numpy as np
import lmp
import glob
import os

#full rw for HPS-T

l = 0.75
temp_dir = f'/home/adria/OPT2/{l}'

delta_l = 0.05
epss = np.linspace(80, 40, 8)

above = lmp.LMP(md_dir=f'/home/adria/perdiux/prod/lammps/final/HPS-T/{l}_New/12D_D4', low_mem=True)
above.rg(full=True)
below = lmp.LMP(md_dir=f'/home/adria/perdiux/prod/lammps/final/HPS-T/{l}_New/D4', low_mem=True)
below.rg(full=True)

> Run Completed at 86.85% for 12D_CPEB4_D4. Ran 868460000 steps for a total of 86847 frames 
> Taking frames every None for a total of [99701, 99701, 94121, 86575, 99701, 99701, 99701, 99701] to avoid strong correlations
> Run Completed at 92.92% for CPEB4_D4. Ran 929240000 steps for a total of 92925 frames 
> Taking frames every None for a total of [99701, 99701, 99701, 99701, 92663, 99701, 99701, 99701] to avoid strong correlations


(array([], dtype=float64), array([], dtype=float64))

In [28]:
T=2
calcer = lmp.LMP(None)
l0, eps0, diff, neff_a, neff_b, rw_A, rw_B = calcer.maximize_charge(above_obj=above,
                       below_obj=below,
                       T=T,
                       l0=0.75,
                       eps0=80,
                       model='HPS-T',
                       temp_dir=temp_dir,)

RUNNING REWEIGHTING
ls=0.75, eps=80, RgA_i=46.283741046975415, RgB_i=45.72660232838287
Rw RgA, 46.919, Rw RgB 46.314, diff 0.60, cost -0.37
Neff-A, 0.13048672409777648 Neff-B, 0.12501073575755248


In [3]:
import numpy as np
s = np.arange(100)

In [10]:
s[-100000:]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [12]:
s==s[-100000:]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [10]:
import mdtraj as md
import MDAnalysis as mda
from MDAnalysis import transformations
import numpy as np
import matplotlib.pyplot as plt

In [2]:
help(transformations)

Help on package MDAnalysis.transformations in MDAnalysis:

NAME
    MDAnalysis.transformations

DESCRIPTION
    Trajectory transformations --- :mod:`MDAnalysis.transformations`
    
    The transformations submodule contains a collection of functions to modify the
    trajectory. Coordinate transformations, such as PBC corrections and molecule fitting
    are often required for some analyses and visualization, and the functions in this
    module allow transformations to be applied on-the-fly.
    These transformation functions can be called by the user for any given
    timestep of the trajectory, added as a workflow using :meth:`add_transformations`
    of the :mod:`~MDAnalysis.coordinates.base` module, or upon Universe creation using
    the keyword argument `transformations`. Note that in the two latter cases, the
    workflow cannot be changed after being defined.
    
    In addition to the specific arguments that each transformation can take, they also
    contain a wrapped func

In [18]:
dcd = '/home/adria/perdiux/prod/lammps/final/HPS-T/0.75/WT/dcd_traj_0.dcd'
lammpstrj = '/home/adria/perdiux/prod/lammps/final/HPS-T/0.75/WT/atom_traj_0.lammpsdump'
top = '/home/adria/perdiux/prod/lammps/final/HPS-T/0.75/WT/topo.pdb'
data = '/home/adria/perdiux/prod/lammps/final/HPS-T/0.75/WT/data.data'
bonds = []
u = mda.Universe(top, lammpstrj)
for i in range(448):
    bonds.append([i,i+1])
del bonds[-1]
u.add_bonds(bonds)
u.bonds
u.trajectory.add_transformations(transformations.unwrap(u.atoms))

/home/adria/anaconda3/lib/python3.7/site-packages/MDAnalysis/topology/PDBParser.py:330: UserWarning: Element information is absent or missing for a few atoms. Elements attributes will not be populated.
  warnings.warn("Element information is absent or missing for a few "


In [19]:
rgs = []
for ts in u.trajectory:
    ts.positions /= 306
    rgs.append(u.atoms.radius_of_gyration())
rgs = np.array(rgs)

In [5]:
rgs.mean()

109.74763168353896

In [6]:
Rgyr = []
for ts in u.trajectory:
    print(ts, u.atoms.radius_of_gyration())

< Timestep 0 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 8913.005029623893
< Timestep 1 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 10292.19265450734
< Timestep 2 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 10781.887429582452
< Timestep 3 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 10628.483974673203
< Timestep 4 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 10682.829922905665
< Timestep 5 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 9858.473901759397
< Timestep 6 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 9350.85037859279
< Timestep 7 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 10676.61427318599
< Timestep 8 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 11729.062415952789
< Timestep 9 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 11393.780965822618
< Timestep 10 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 12172.698057

< Timestep 130 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 14709.787600590062
< Timestep 131 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 14763.573065992476
< Timestep 132 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 15437.308274392772
< Timestep 133 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 15679.0026780904
< Timestep 134 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 15469.866844472439
< Timestep 135 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 15501.633166284735
< Timestep 136 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 14983.938998621015
< Timestep 137 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 13670.913389087875
< Timestep 138 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 13238.78541076951
< Timestep 139 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 12855.850306161401
< Timestep 140 with unit cell dimensions [306. 306. 306.  90.  

< Timestep 260 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 14309.834957955933
< Timestep 261 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 14133.65897845207
< Timestep 262 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 14713.232536131014
< Timestep 263 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 16005.674855005402
< Timestep 264 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 14591.854377481184
< Timestep 265 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 15534.5586183295
< Timestep 266 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 14182.488067580492
< Timestep 267 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 12975.281916500402
< Timestep 268 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 13744.838527118427
< Timestep 269 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 13780.327008269991
< Timestep 270 with unit cell dimensions [306. 306. 306.  90.  

< Timestep 389 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 15833.72572778077
< Timestep 390 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 17678.954430813872
< Timestep 391 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 17584.985811190007
< Timestep 392 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 16104.59174134333
< Timestep 393 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 14284.26519962666
< Timestep 394 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 13245.890187553248
< Timestep 395 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 11786.698477975044
< Timestep 396 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 12051.326552969322
< Timestep 397 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 12676.619268621402
< Timestep 398 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 20478.435326616567
< Timestep 399 with unit cell dimensions [306. 306. 306.  90.  

< Timestep 516 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 12258.777025167588
< Timestep 517 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 12966.92422235178
< Timestep 518 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 12745.86362447541
< Timestep 519 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 12864.848951523756
< Timestep 520 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 13265.0441706697
< Timestep 521 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 14603.34197569094
< Timestep 522 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 13966.835164882266
< Timestep 523 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 13663.343613333309
< Timestep 524 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 14482.044502242565
< Timestep 525 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 15141.52529366256
< Timestep 526 with unit cell dimensions [306. 306. 306.  90.  90.

...

< Timestep 1552 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 29509.137393399753
< Timestep 1553 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 28572.20564515405
< Timestep 1554 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 28541.767974070553
< Timestep 1555 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 28505.23028775862
< Timestep 1556 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 19104.77925379743
< Timestep 1557 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 20026.566264739085
< Timestep 1558 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 16508.16038257503
< Timestep 1559 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 11047.740134239233
< Timestep 1560 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 22507.15292198224
< Timestep 1561 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 12498.88402549368
< Timestep 1562 with unit cell dimensions [306. 306. 306

< Timestep 1681 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 41062.49663927585
< Timestep 1682 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 38098.333010183094
< Timestep 1683 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 39094.527462659
< Timestep 1684 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 41206.664398171015
< Timestep 1685 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 40442.370769468835
< Timestep 1686 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 40794.32090808699
< Timestep 1687 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 37093.01350861309
< Timestep 1688 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 36150.862056791564
< Timestep 1689 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 41919.753706318494
< Timestep 1690 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 37948.565483819555
< Timestep 1691 with unit cell dimensions [306. 306. 306

< Timestep 1809 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 28629.953604602233
< Timestep 1810 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 29244.842013327503
< Timestep 1811 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 29579.56534920367
< Timestep 1812 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 30233.078526657617
< Timestep 1813 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 28001.20793980973
< Timestep 1814 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 18811.997768440386
< Timestep 1815 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 15063.594552778395
< Timestep 1816 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 21291.35470262342
< Timestep 1817 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 21030.614737804033
< Timestep 1818 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 22769.437149474572
< Timestep 1819 with unit cell dimensions [306. 306. 

< Timestep 1938 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 10931.014817840254
< Timestep 1939 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 10637.19204367158
< Timestep 1940 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 10220.928145189426
< Timestep 1941 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 11175.542846595179
< Timestep 1942 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 10981.951992233344
< Timestep 1943 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 10684.715289301259
< Timestep 1944 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 10876.26285457454
< Timestep 1945 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 11135.743207800757
< Timestep 1946 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 10952.849238216468
< Timestep 1947 with unit cell dimensions [306. 306. 306.  90.  90.  90.] > 11930.936149514568
< Timestep 1948 with unit cell dimensions [306. 306.

KeyboardInterrupt: 

In [4]:
np.mean(Rgyr)

30652.772702602695

In [2]:
s = ["WT", "D4", "7D_WT", "12D_D4"]
for prot in s:
    for i in range(7):
        print(f"Doing {i} {prot}")
        try: 
            md.load(f'/home/adria/perdiux/prod/lammps/final/HPS-T/0.75/{prot}/dcd_traj_{i}.dcd', top=f'/home/adria/perdiux/prod/lammps/final/HPS-T/0.75/{prot}/topo.pdb')
        except:
            print(f"Failed for {i} {prot}")

Doing 0 WT
Doing 1 WT
Failed for 1 WT
Doing 2 WT
Failed for 2 WT
Doing 3 WT
Failed for 3 WT
Doing 4 WT
Doing 5 WT
Doing 6 WT
Doing 0 D4
Doing 1 D4
Doing 2 D4
Doing 3 D4
Failed for 3 D4
Doing 4 D4
Failed for 4 D4
Doing 5 D4
Failed for 5 D4
Doing 6 D4
Doing 0 7D_WT
Failed for 0 7D_WT
Doing 1 7D_WT
Failed for 1 7D_WT
Doing 2 7D_WT
Failed for 2 7D_WT
Doing 3 7D_WT
Failed for 3 7D_WT
Doing 4 7D_WT
Failed for 4 7D_WT
Doing 5 7D_WT
Failed for 5 7D_WT
Doing 6 7D_WT
Doing 0 12D_D4
Doing 1 12D_D4
Doing 2 12D_D4
Doing 3 12D_D4
Doing 4 12D_D4
Failed for 4 12D_D4
Doing 5 12D_D4
Failed for 5 12D_D4
Doing 6 12D_D4


In [28]:
rg = md.compute_rg(r)

In [25]:
rg.mean()*10.

56.478165708175624

In [15]:
for ts in u.trajectory:
    print("Frame: {0:5d}, Time: {1:8.3f} ps".format(ts.frame, u.trajectory.time))
    print("Rgyr: {0:g} A".format(u.atoms.radius_of_gyration()))

Frame:     0, Time:    0.000 ps
Rgyr: 29.1275 A
Frame:     1, Time: 4888.821 ps
Rgyr: 32.4932 A
Frame:     2, Time: 9777.642 ps
Rgyr: 35.5155 A
Frame:     3, Time: 14666.463 ps
Rgyr: 37.2803 A
Frame:     4, Time: 19555.284 ps
Rgyr: 37.474 A
Frame:     5, Time: 24444.105 ps
Rgyr: 35.2506 A
Frame:     6, Time: 29332.926 ps
Rgyr: 31.9805 A
Frame:     7, Time: 34221.747 ps
Rgyr: 33.945 A
Frame:     8, Time: 39110.568 ps
Rgyr: 37.6788 A
Frame:     9, Time: 43999.389 ps
Rgyr: 40.1761 A
Frame:    10, Time: 48888.210 ps
Rgyr: 41.5277 A
Frame:    11, Time: 53777.031 ps
Rgyr: 38.8142 A
Frame:    12, Time: 58665.852 ps
Rgyr: 42.7697 A
Frame:    13, Time: 63554.673 ps
Rgyr: 43.9003 A
Frame:    14, Time: 68443.494 ps
Rgyr: 44.9167 A
Frame:    15, Time: 73332.315 ps
Rgyr: 43.913 A
Frame:    16, Time: 78221.136 ps
Rgyr: 46.0565 A
Frame:    17, Time: 83109.957 ps
Rgyr: 49.4143 A
Frame:    18, Time: 87998.778 ps
Rgyr: 50.3858 A
Frame:    19, Time: 92887.599 ps
Rgyr: 52.4159 A
Frame:    20, Time: 97776.

Rgyr: 38.6 A
Frame:   177, Time: 865321.317 ps
Rgyr: 38.3569 A
Frame:   178, Time: 870210.138 ps
Rgyr: 36.659 A
Frame:   179, Time: 875098.959 ps
Rgyr: 34.9645 A
Frame:   180, Time: 879987.780 ps
Rgyr: 37.8225 A
Frame:   181, Time: 884876.601 ps
Rgyr: 42.7532 A
Frame:   182, Time: 889765.422 ps
Rgyr: 45.1911 A
Frame:   183, Time: 894654.243 ps
Rgyr: 43.7868 A
Frame:   184, Time: 899543.064 ps
Rgyr: 40.9614 A
Frame:   185, Time: 904431.885 ps
Rgyr: 38.6528 A
Frame:   186, Time: 909320.706 ps
Rgyr: 35.1651 A
Frame:   187, Time: 914209.527 ps
Rgyr: 37.3847 A
Frame:   188, Time: 919098.348 ps
Rgyr: 37.8864 A
Frame:   189, Time: 923987.169 ps
Rgyr: 39.3404 A
Frame:   190, Time: 928875.990 ps
Rgyr: 45.3699 A
Frame:   191, Time: 933764.811 ps
Rgyr: 45.5956 A
Frame:   192, Time: 938653.632 ps
Rgyr: 42.5093 A
Frame:   193, Time: 943542.453 ps
Rgyr: 46.4643 A
Frame:   194, Time: 948431.274 ps
Rgyr: 47.1753 A
Frame:   195, Time: 953320.095 ps
Rgyr: 48.686 A
Frame:   196, Time: 958208.916 ps
Rgyr:

Rgyr: 40.3339 A
Frame:   354, Time: 1730642.634 ps
Rgyr: 41.9303 A
Frame:   355, Time: 1735531.455 ps
Rgyr: 42.0096 A
Frame:   356, Time: 1740420.276 ps
Rgyr: 45.0261 A
Frame:   357, Time: 1745309.097 ps
Rgyr: 43.9521 A
Frame:   358, Time: 1750197.918 ps
Rgyr: 42.2444 A
Frame:   359, Time: 1755086.739 ps
Rgyr: 42.0566 A
Frame:   360, Time: 1759975.560 ps
Rgyr: 37.92 A
Frame:   361, Time: 1764864.381 ps
Rgyr: 38.3741 A
Frame:   362, Time: 1769753.202 ps
Rgyr: 38.0362 A
Frame:   363, Time: 1774642.023 ps
Rgyr: 35.859 A
Frame:   364, Time: 1779530.844 ps
Rgyr: 36.145 A
Frame:   365, Time: 1784419.665 ps
Rgyr: 36.2849 A
Frame:   366, Time: 1789308.486 ps
Rgyr: 36.6803 A
Frame:   367, Time: 1794197.307 ps
Rgyr: 33.1041 A
Frame:   368, Time: 1799086.128 ps
Rgyr: 32.2156 A
Frame:   369, Time: 1803974.949 ps
Rgyr: 32.8766 A
Frame:   370, Time: 1808863.770 ps
Rgyr: 33.2692 A
Frame:   371, Time: 1813752.591 ps
Rgyr: 35.809 A
Frame:   372, Time: 1818641.412 ps
Rgyr: 36.9962 A
Frame:   373, Time: 

Rgyr: 63.9811 A
Frame:   536, Time: 2620408.056 ps
Rgyr: 63.2837 A
Frame:   537, Time: 2625296.877 ps
Rgyr: 66.4977 A
Frame:   538, Time: 2630185.698 ps
Rgyr: 67.2921 A
Frame:   539, Time: 2635074.519 ps
Rgyr: 72.126 A
Frame:   540, Time: 2639963.340 ps
Rgyr: 73.4018 A
Frame:   541, Time: 2644852.161 ps
Rgyr: 75.472 A
Frame:   542, Time: 2649740.982 ps
Rgyr: 77.661 A
Frame:   543, Time: 2654629.803 ps
Rgyr: 75.2721 A
Frame:   544, Time: 2659518.624 ps
Rgyr: 72.9899 A
Frame:   545, Time: 2664407.445 ps
Rgyr: 69.2656 A
Frame:   546, Time: 2669296.266 ps
Rgyr: 69.8029 A
Frame:   547, Time: 2674185.087 ps
Rgyr: 75.7328 A
Frame:   548, Time: 2679073.908 ps
Rgyr: 74.806 A
Frame:   549, Time: 2683962.729 ps
Rgyr: 77.1341 A
Frame:   550, Time: 2688851.550 ps
Rgyr: 76.0517 A
Frame:   551, Time: 2693740.371 ps
Rgyr: 73.9344 A
Frame:   552, Time: 2698629.192 ps
Rgyr: 74.2288 A
Frame:   553, Time: 2703518.013 ps
Rgyr: 68.8846 A
Frame:   554, Time: 2708406.834 ps
Rgyr: 65.9642 A
Frame:   555, Time:

Rgyr: 62.1498 A
Frame:   718, Time: 3510173.478 ps
Rgyr: 60.6313 A
Frame:   719, Time: 3515062.299 ps
Rgyr: 55.8483 A
Frame:   720, Time: 3519951.120 ps
Rgyr: 59.7532 A
Frame:   721, Time: 3524839.941 ps
Rgyr: 58.6175 A
Frame:   722, Time: 3529728.762 ps
Rgyr: 62.6934 A
Frame:   723, Time: 3534617.583 ps
Rgyr: 58.9194 A
Frame:   724, Time: 3539506.404 ps
Rgyr: 57.5262 A
Frame:   725, Time: 3544395.225 ps
Rgyr: 55.3355 A
Frame:   726, Time: 3549284.046 ps
Rgyr: 58.9411 A
Frame:   727, Time: 3554172.867 ps
Rgyr: 62.2029 A
Frame:   728, Time: 3559061.688 ps
Rgyr: 64.3082 A
Frame:   729, Time: 3563950.509 ps
Rgyr: 66.3744 A
Frame:   730, Time: 3568839.330 ps
Rgyr: 62.2998 A
Frame:   731, Time: 3573728.151 ps
Rgyr: 64.0155 A
Frame:   732, Time: 3578616.972 ps
Rgyr: 64.7356 A
Frame:   733, Time: 3583505.793 ps
Rgyr: 61.688 A
Frame:   734, Time: 3588394.614 ps
Rgyr: 66.4079 A
Frame:   735, Time: 3593283.435 ps
Rgyr: 60.4667 A
Frame:   736, Time: 3598172.256 ps
Rgyr: 59.1407 A
Frame:   737, Ti

...

Rgyr: 73.0073 A
Frame: 16042, Time: 78426466.482 ps
Rgyr: 75.7905 A
Frame: 16043, Time: 78431355.303 ps
Rgyr: 76.679 A
Frame: 16044, Time: 78436244.124 ps
Rgyr: 84.8895 A
Frame: 16045, Time: 78441132.945 ps
Rgyr: 88.2432 A
Frame: 16046, Time: 78446021.766 ps
Rgyr: 82.1336 A
Frame: 16047, Time: 78450910.587 ps
Rgyr: 84.5537 A
Frame: 16048, Time: 78455799.408 ps
Rgyr: 84.171 A
Frame: 16049, Time: 78460688.229 ps
Rgyr: 88.3902 A
Frame: 16050, Time: 78465577.050 ps
Rgyr: 89.5571 A
Frame: 16051, Time: 78470465.871 ps
Rgyr: 89.351 A
Frame: 16052, Time: 78475354.692 ps
Rgyr: 85.6305 A
Frame: 16053, Time: 78480243.513 ps
Rgyr: 84.4129 A
Frame: 16054, Time: 78485132.334 ps
Rgyr: 92.3737 A
Frame: 16055, Time: 78490021.155 ps
Rgyr: 92.2107 A
Frame: 16056, Time: 78494909.976 ps
Rgyr: 88.0869 A
Frame: 16057, Time: 78499798.797 ps
Rgyr: 84.033 A
Frame: 16058, Time: 78504687.618 ps
Rgyr: 88.2962 A
Frame: 16059, Time: 78509576.439 ps
Rgyr: 90.3094 A
Frame: 16060, Time: 78514465.260 ps
Rgyr: 89.8246 A


Rgyr: 64.2958 A
Frame: 16225, Time: 79321120.725 ps
Rgyr: 62.1526 A
Frame: 16226, Time: 79326009.546 ps
Rgyr: 59.1556 A
Frame: 16227, Time: 79330898.367 ps
Rgyr: 58.6625 A
Frame: 16228, Time: 79335787.188 ps
Rgyr: 60.1817 A
Frame: 16229, Time: 79340676.009 ps
Rgyr: 58.3551 A
Frame: 16230, Time: 79345564.830 ps
Rgyr: 54.3583 A
Frame: 16231, Time: 79350453.651 ps
Rgyr: 54.4513 A
Frame: 16232, Time: 79355342.472 ps
Rgyr: 52.5882 A
Frame: 16233, Time: 79360231.293 ps
Rgyr: 51.4473 A
Frame: 16234, Time: 79365120.114 ps
Rgyr: 53.4059 A
Frame: 16235, Time: 79370008.935 ps
Rgyr: 54.3476 A
Frame: 16236, Time: 79374897.756 ps
Rgyr: 55.7186 A
Frame: 16237, Time: 79379786.577 ps
Rgyr: 57.4689 A
Frame: 16238, Time: 79384675.398 ps
Rgyr: 64.6559 A
Frame: 16239, Time: 79389564.219 ps
Rgyr: 66.5363 A
Frame: 16240, Time: 79394453.040 ps
Rgyr: 66.2108 A
Frame: 16241, Time: 79399341.861 ps
Rgyr: 66.9249 A
Frame: 16242, Time: 79404230.682 ps
Rgyr: 71.0913 A
Frame: 16243, Time: 79409119.503 ps
Rgyr: 72.151

Rgyr: 46.3032 A
Frame: 16408, Time: 80215774.968 ps
Rgyr: 47.2711 A
Frame: 16409, Time: 80220663.789 ps
Rgyr: 45.3875 A
Frame: 16410, Time: 80225552.610 ps
Rgyr: 47.3698 A
Frame: 16411, Time: 80230441.431 ps
Rgyr: 46.5368 A
Frame: 16412, Time: 80235330.252 ps
Rgyr: 46.8262 A
Frame: 16413, Time: 80240219.073 ps
Rgyr: 50.0267 A
Frame: 16414, Time: 80245107.894 ps
Rgyr: 49.0625 A
Frame: 16415, Time: 80249996.715 ps
Rgyr: 47.4224 A
Frame: 16416, Time: 80254885.536 ps
Rgyr: 48.4672 A
Frame: 16417, Time: 80259774.357 ps
Rgyr: 51.356 A
Frame: 16418, Time: 80264663.178 ps
Rgyr: 47.64 A
Frame: 16419, Time: 80269551.999 ps
Rgyr: 49.6952 A
Frame: 16420, Time: 80274440.820 ps
Rgyr: 48.9195 A
Frame: 16421, Time: 80279329.641 ps
Rgyr: 47.0536 A
Frame: 16422, Time: 80284218.462 ps
Rgyr: 43.6773 A
Frame: 16423, Time: 80289107.283 ps
Rgyr: 42.2055 A
Frame: 16424, Time: 80293996.104 ps
Rgyr: 45.2727 A
Frame: 16425, Time: 80298884.925 ps
Rgyr: 46.6417 A
Frame: 16426, Time: 80303773.746 ps
Rgyr: 46.2641 A

Rgyr: 41.0955 A
Frame: 16590, Time: 81105540.390 ps
Rgyr: 42.168 A
Frame: 16591, Time: 81110429.211 ps
Rgyr: 36.8999 A
Frame: 16592, Time: 81115318.032 ps
Rgyr: 36.7229 A
Frame: 16593, Time: 81120206.853 ps
Rgyr: 35.064 A
Frame: 16594, Time: 81125095.674 ps
Rgyr: 34.0797 A
Frame: 16595, Time: 81129984.495 ps
Rgyr: 34.7004 A
Frame: 16596, Time: 81134873.316 ps
Rgyr: 35.3587 A
Frame: 16597, Time: 81139762.137 ps
Rgyr: 35.298 A
Frame: 16598, Time: 81144650.958 ps
Rgyr: 34.1416 A
Frame: 16599, Time: 81149539.779 ps
Rgyr: 35.8196 A
Frame: 16600, Time: 81154428.600 ps
Rgyr: 38.317 A
Frame: 16601, Time: 81159317.421 ps
Rgyr: 36.8325 A
Frame: 16602, Time: 81164206.242 ps
Rgyr: 38.0324 A
Frame: 16603, Time: 81169095.063 ps
Rgyr: 37.7749 A
Frame: 16604, Time: 81173983.884 ps
Rgyr: 40.0458 A
Frame: 16605, Time: 81178872.705 ps
Rgyr: 36.372 A
Frame: 16606, Time: 81183761.526 ps
Rgyr: 35.6653 A
Frame: 16607, Time: 81188650.347 ps
Rgyr: 37.8721 A
Frame: 16608, Time: 81193539.168 ps
Rgyr: 33.0543 A
F

KeyboardInterrupt: 

In [9]:
r = md.load('/home/adria/perdiux/prod/lammps/final/HPS-T/0.75/WT/atom_traj_0.lammpstrj', top='/home/adria/perdiux/prod/lammps/final/HPS-T/0.75/WT/topo.pdb')

In [11]:
md.compute_rg(r).mean()

10.97476316868513

In [10]:
r2 = md.load('/home/adria/perdiux/prod/lammps/final/HPS-T/0.75/WT/dcd_traj_0.dcd', top='/home/adria/perdiux/prod/lammps/final/HPS-T/0.75/WT/topo.pdb')

In [12]:
md.compute_rg(r2).mean()

5.647816570817563